In [ ]:
from aim2dat.strct import StructureCollection

strct_c = StructureCollection()

The structures can be added to the object via the different `append*` functions of the object:

In [ ]:
from ase.spacegroup import crystal
import aiida
from aiida.orm import StructureData
from aim2dat.strct import Structure

structure_dict = {
    "label": "Benzene",
    "elements": ["C"] * 6 + ["H"] * 6,
    "pbc": False,
    "positions": [
        [-0.7040, -1.2194, -0.0000],
        [0.7040, -1.2194, -0.0000],
        [-1.4081, -0.0000, -0.0000],
        [1.4081, 0.0000, 0.0000],
        [-0.7040, 1.2194, 0.0000],
        [0.7040, 1.2194, -0.0000],
        [-1.2152, -2.1048, -0.0000],
        [1.2152, -2.1048, 0.0000],
        [-2.4304, -0.0000, 0.0000],
        [2.4304, 0.0000, 0.0000],
        [-1.2152, 2.1048, -0.0000],
        [1.2152, 2.1048, 0.0000],
    ],
}
strct_c.append(**structure_dict)

structure = Structure(
    elements=["O", "H", "H"],
    positions=[[0.0, 0.0, 0.119], [0.0, 0.763, -0.477], [0.0, -0.763, -0.477]],
    pbc=False,
)
strct_c.append_structure(structure, label="Water")

a = 4.066 * 2.0
GaAs_conv = crystal(
    ("Ga", "As"),
    basis=((0.0, 0.0, 0.0), (0.75, 0.75, 0.75)),
    spacegroup=216,
    cellpar=[a, a, a, 90, 90, 90],
    primitive_cell=False,
)
strct_c.append_from_ase_atoms("GaAs", GaAs_conv)

aiida.load_profile("tests")
unit_cell = [[3.0, 0.0, 0.0], [0.0, 3.0, 0.0], [0.0, 0.0, 3.0]]
structure = StructureData(cell=unit_cell)
structure.label = "Li"
structure.append_atom(position=(0.0, 0.0, 0.0), symbols="Li")
structure.append_atom(position=(1.5, 1.5, 1.5), symbols="Li")
strct_c.append_from_aiida_structuredata(structure)

Alternatively, a list of dictionaries can be passed upon initialization of the object:

In [ ]:
strct_c2 = StructureCollection(structures=[structure_dict])

A summary of the object is given by its string representation:

In [ ]:
print(strct_c)

Additionally, a pandas data frame can be created based on the object's content:

In [ ]:
strct_c2.create_pandas_df()

In [ ]:
print(strct_c[1])

In [ ]:
print(strct_c["Water"])

In [ ]:
print(strct_c.get_structure(3))

In [ ]:
del strct_c["Benzene"]
strct_c.labels

Two structure collection objects can be merged into one using `+`:

In [ ]:
print(strct_c + strct_c2)

In [ ]:
strct_c.store_in_aiida_db(group_label="test")
strct_c = StructureCollection()
strct_c.import_from_aiida_db(group_label="test")

In [ ]:
strct_c.store_in_hdf5_file("test.h5")
strct_c = StructureCollection()
strct_c.import_from_hdf5_file("test.h5")

In [ ]:
from aim2dat.strct import StructureOperations

strct_c += strct_c2
strct_op = StructureOperations(structures=strct_c, verbose=False)

In [ ]:
strct_op.supported_output_formats

In [ ]:
print("Analysis methods: ", strct_op.analysis_methods)
print("Manipulation methods: ", strct_op.manipulation_methods)

In [ ]:
print("One structure: ", strct_op[["Benzene"]].calculate_distance(2, 3))
print("One structure: ", strct_c["Benzene"].calculate_distance(site_index1=2, site_index2=3))

In [ ]:
print("Multiple structures: ", strct_op[0,1].calculate_distance(0, 1))

In [ ]:
strct_op.output_format = "DataFrame"
strct_op[0, 1].calculate_distance(0, 1)

In [ ]:
subst_structure = strct_op["GaAs"].substitute_elements(("Ga", "Al"), change_label=True)
print(subst_structure)
print(strct_op.structures.labels)

In [ ]:
subst_structure = strct_op["GaAs"].substitute_elements(("Ga", "Al"), change_label=False)
print(subst_structure)
print(strct_op.structures.labels)
print(strct_op.structures["GaAs"])

In [ ]:
subst_structures = strct_op[strct_op.structures.labels].substitute_elements(
    ("Al", "Ga"), change_label=False
)
print(subst_structures)

In [ ]:
from aim2dat.strct.ext_analysis import calculate_prdf

output = strct_op["Benzene"].perform_analysis(calculate_prdf, {"r_max": 7.5})